![image.png](plots/dim_red_intro.png)

En los problemas de machine learning es habitual trabajar con decenas, cientos o incluso miles de variables simultaneamente a la hora de hacer predicciones. Cuantas más variables tengamos, mejor, ¿no? Más información tendremos disponible... __Cuidado, porque eso no tiene porque ser cierto__. Puede que muchas de las variables del conjunto nos den información redundante (por ejemplo, código postal y ciudad), o puede que la información que nos están dando simplemente no sea relevante para nuestro problema (por ejemplo, el color de ojos para predecir la morosidad). Todas esas variables extra lo único que hacen es ocupar memoria en el ordenador, ralentizar la ejecución de los algoritmos y, en última instancia, producir predicciones que estén sobre ajustadas.

Esto es lo que se conoce como __la maldición de la dimensionalidad__, y la forma de luchar contra este problema es... reduciendo la dimensión. Existen varias formas de reducir la dimensión de un conjunto de datos. Aquí nos centraremos en dos alternativas:

* Selección de variables

* Proyecciones

# Selección de variables con filtering
___

La idea intuitiva detrás de la selección de variables es sencilla: intentar seleccionar las variables más importantes de nuestro conjunto, y descartar las demás. Aquí nos centraremos en la selección con filtering:

* Filtering: Hacemos un ranking de importancia de las variables y filtramos para quedarnos con las primeras del ranking. Después ejecutamos nuestro modelo con este subconjunto de variables.

## BostonHousing
Para ello, veamos un ejemplo. Nos descargaremos el conjunto de datos de __Boston Housing__, que contiene información sobre el precio mediano de la vivienda en diferentes barrios de la ciudad de boston, así como una serie de variables socio-económicas de esos barrios. El objetivo de este problema es predecir el precio mediano de la vivienda en un barrio dadas el resto de variables. Empecemos descargando los datos.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.datasets import load_boston

%matplotlib inline

np.random.seed(5) # Los resultados del notebook serán los mismos en cada ejecución

In [ ]:
boston = load_boston()
X_boston = pd.DataFrame(boston.data, columns = boston.feature_names)

y_boston = boston.target

In [ ]:
X_boston.describe()

In [ ]:
p = plt.hist(y_boston, bins=50)

### <font color='D12828'> Ejercicio: </font>
1. Obtén un scatterplot entre la variable respuesta y cada una de las variables predictoras. Pista: Usa `plt.subplot` y un bucle `for`.

In [8]:
# Ejercicio


### <font color='D12828'> Ejercicio: </font>
1. Divide el conjunto de datos en `X_boston_train`, `y_boston_train`, `X_boston_test`, `y_boston_test` usando un 80% de los datos para training. Pista: `from sklearn.model_selection import train_test_split`

2. Estandariza el conjunto de datos de train y de test y llámalo `X_boston_train_s` y `X_boston_test_s`. Pista: `from sklearn.preprocessing import StandardScaler`

In [ ]:
# EJercicio


Entrenemos un modelo con todas las variables para tomarlo de referencia. Para ello usaremos una SVM de regresión, ya que la variable respuesta es cuantitativa.

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import os

num_cores = os.cpu_count()

# Entrenar modelo
svr_model = SVR(kernel='linear')
svr_model.fit(X_boston_train_s, y_boston_train)

# Predecir en el conjunto de test
y_test_pred = svr_model.predict(X_boston_test_s)

# Error 
error = mean_squared_error(y_boston_test, y_test_pred)
print(f'Error cuadrático medio: {error}')

## `SelectKBest` en regresión

Esta función forma parte del paquete de sklearn. Su funcionamiento es sencillo: __selecciona las k mejores variables__ en función del criterio que nosotros le indiquemos. Lo habitual en problemas de regresión es utilizar como criterio el `f_regression`, que está muy relacionado con la correlación entre las variables predictoras y la respuesta, aunque hay otras alternativas, como `mutual_info_regression`, especialmente recomendada si creemos que la relación entre las variables y la respuesta puede ser no lineal. Podeis consultar toda la información de esta función [aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest)

![image.png](plots/f_mi.png)

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(f_regression, k="all")
selector.fit(X_boston_train_s, y_boston_train)

Las métricas de cada variable se almacenan en `selector.scores_`. Representemoslas gráficamente

In [ ]:
plt.figure(figsize=(14, 6))
plt.bar(boston.feature_names, selector.scores_)

Podemos ver que las variables más importantes son `LSTAT` y `RM`. Así que seleccionaremos estas dos variables y construiremos un nuevo modelo solo con ellas. Esto reducirá la dimensión desde las 13 variables iniciales a 2.

In [ ]:
# Seleccionar variables en train
selector = SelectKBest(f_regression, k=2)
X_train_reduced = selector.fit_transform(X_boston_train_s, y_boston_train)

# Seleccionar variables en test
X_test_reduced = selector.transform(X_boston_test_s)

svr_model.fit(X_train_reduced, y_boston_train)

# Predecir en el conjunto de test
y_test_pred = svr_model.predict(X_test_reduced)

# Error 
error = mean_squared_error(y_boston_test, y_test_pred)
print(f'Error cuadrático medio: {error}')

### <font color='D12828'> Ejercicio: </font>
1. Construye una SVM utilizando las 4 mejores variables de acuerdo al criterio de `mutual_info_regression`
2. Obtén el valor del error cuadrático medio en el conjunto de test

In [ ]:
# Ejercicio


### <font color='D12828'> Ejercicio: </font>

1. Construye un pipeline que:
  1. Estandarice los datos
  2. Llame a SelectKBest usando el criterio de f_regression
  3. Construya una SVM
2. Usa la función de `GridSearchCV` para optimizar el número de variables seleccionadas de tu pipeline. Pista: `from sklearn.pipeline import Pipeline`, `from sklearn.model_selection import GridSearchCV`

In [ ]:
# Ejercicio


## Breast cancer wisconsin

El conjunto de datos de Breast cancer wisconsin está formado por 569 observaciones de pacientes con un tumor, en los cuales se midieron diferentes aspectos de la forma y tamaño del tumor, que conforman 30 variables numéricas. La variable respuesta es categórica, e indica si el tumor es benigno o maligno. El objetivo es predecir el tipo de tumor usando las variables del conjunto de datos. Empecemos observando descargando los datos y observando si las dos categorías están balanceadas:

In [ ]:
from sklearn.datasets import load_breast_cancer

bc = load_breast_cancer()

X_bc, y_bc = bc.data, bc.target

value, count = np.unique(y_bc, return_counts=True)
plt.bar(value, count)

## `SelectKBest` en clasificación

Hemos visto cómo hacer una selección de variables si trabajamos en un problema de regresión, en el que la variable respuesta sea cuantitativa. También podemos hacer el mismo proceso en problemas de clasificación, utilizándo métricas especiales para clasificación. Se usan especialmente las métricas de `f_classif` y de `mutual_info_classif`. Para variar de las SVM, utilizaremos el algoritmo de los K vecinos más próximos `KNeighborsClassifier` en este ejemplo. 


In [ ]:
X_bc_train, X_bc_test, y_bc_train, y_bc_test = train_test_split(X_bc, y_bc, test_size=0.2, random_state=42, stratify=y_bc)

scaler = StandardScaler()
X_bc_train_s = scaler.fit_transform(X_bc_train)
X_bc_test_s = scaler.fit_transform(X_bc_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Entrenar modelo
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(X_bc_train_s, y_bc_train)

# Predecir en el conjunto de test
y_test_pred = knn.predict(X_bc_test_s)

# Error 
error = accuracy_score(y_bc_test, y_test_pred)
print(f'Accuracy: {error}')

### <font color='D12828'> Ejercicio: </font>
1. Usa `SelectKBest` y representa gráficamente la importancia de las variables de acuerdo al test `f_classif`
2. Construye una SVM con el número de variables que consideres óptimo en base al gráfico

In [ ]:
# Ejercicio


# Análisis de componentes principales (PCA)
___

El PCA es la técnica de reducción de la dimensión más utilizada en el mundo. La idea básica detrás del PCA consiste en utilizar las variables originales para __construir unas nuevas variables__, de forma que estas nuevas variables, llamadas componentes principales, expliquen la __mayor cantidad de información__ posible pero sean __ortogonales__ entre sí. Al ser ortogonales entre si, no comparten información por lo que con un número pequeño de componentes principales, se puede explicar la gran mayoría de la información del conjunto de datos original.

Matemáticamene, buscamos encontrar una matriz $P$ tal que $XP=T$, donde $X$ es la matriz con las variables originales, y $T$ es la matriz con las nuevas variables. Veamos un ejemplo con el conjunto de datos de iris. Empecemos sin reducir dimensión, calculando 4 componentes.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris

iris = load_iris()
X_iris, y_iris = iris.data, iris.target

In [ ]:
pca_iris = PCA(n_components=4)

T_iris = pca_iris.fit_transform(X_iris)  # scores
P_iris = pca_iris.components_.T          # loadings

P_iris = pd.DataFrame(P_iris, index=iris.feature_names, columns=['PC1', 'PC2', 'PC3', 'PC4'])
P_iris

Cada componente principal no es más que un vector apuntando en la dirección de la nueva variable. Como dos vectores con signo opuesto están en el mismo eje aunque tengan direcciones opuestas, __el signo de las componentes principales no es estable__, aunque el valor de los coeficientes sí lo sea. Además, las componentes pueden ser interpretadas (en función de nuestro conocimiento previo de las variables originales). Por ejemplo, 

* La segunda componente, PC2 tiene valores grandes para _sepal length_ y _sepal width_, por lo que es una variable que agrupa la información acerca de la forma del sépalo. A mayor sepalo, mayor será PC2.
* La tercera componente, PC3 tiene un peso negativo para _sepal length_, y positivo para _sepal_width_ y _petal_width_. Esta variable tendrá valores grandes si el petalo y el sépalo son anchos, y el sépalo no es muy largo.

## Comprobación matricial 

Para hacer PCA es necesario trabajar con los datos centrados. Usando sklearn, el algoritmo los centra automáticamente. Hemos dicho que $XP=T$. Veamos si esto es cierto:

In [ ]:
# Centramos los datos
X_s = X_iris - np.mean(X_iris, axis=0)

# Visualizamos la primera fila de la matriz centrada
print(f'Xs: {X_s[0,]}')

In [ ]:
# Calculamos el producto de XP
print(f'T: {T_iris[0,]}\n')
print(f'XP={np.dot(X_s, P_iris)[0,:]}')


De la misma forma, dadas $T$ y $P$ podemos reconstruir $X$: $X=TP^t$

In [ ]:
print(f'Xs : {X_s[0,]}\n')
print(f'TP.T={np.dot(T_iris, P_iris.T)[0,:]}')

### <font color='D12828'> Ejercicio: </font>
1. Utilizando el conjunto de datos de cancer de mama `X_bc`, obtén las 4 primeras componentes principales.

In [ ]:
# Ejercicio


## Variabilidad explicada

Habíamos dicho que las componentes principales se conatruían de forma que cada una explicaba la mayor cantidad de información posible, siendo ortogonales entre sí (es decir, la información que cada variable proporciona es diferente del resto) ¿Pero cuanta información proporciona cada variable? Eso se almacena en `pca.explained_variance_ratio_`:

In [ ]:
pca_iris.explained_variance_ratio_

In [ ]:
np.cumsum(pca_iris.explained_variance_ratio_)

Esto nos indica que la primera variable proporciona un 92% de la información original, la segunda variable un 5%, la tercera un 1.7% y la última únicamente proporciona un 0.5%. Básicamente, si en la matriz `T_pca` seleccionasemos las dos primeras columnas, pasaríamos a tener un conjunto de datos de dos dimensiones que explica el 97% de la variabilidad original. Podríamos reducir la dimensión a la mitad sin perder prácticamente nada de información. 

### <font color='D12828'> Ejercicio: </font>
1. Obtén la variabilidad explicada por cada componente principal calculada para el conjunto de datos de cancer en el ejercicio anterior
2. Haz un scatterplot de las dos primeras componentes principales. Haz que el color cambie en función del tipo de tumor (almacenado en `y_bc`)

In [ ]:
# Ejercicio


## ¿Afecta la escala de las variables?

### <font color='D12828'> Ejercicio: </font>

1. Obtén las dos primeras componentes principales del conjunto de datos de cancer escalado `X_bc_train`
2. Haz un scatterplot de las dos primeras componentes principales. Haz que el color cambie en función del tipo de tumor (almacenado en y_bc_train)

In [ ]:
# Ejercicio


## ¿Con cuantas componentes nos quedamos?

Si nuestra matriz de datos original $X$ tiene $n$ observaciones y $p$ variables, podremos calcular como máximo, min(n, p) componentes. Eso quiere decir que:

* Si tenemos 500 filas y 10 variables, como máximo podremos obtener __10 componentes principales__, y las 10 componentes explicarán el 100% de la variabilidad original.
* Si tenemos 100 filas y 2000 variables, como máximo podremos obtener __100 componentes principales__, que explicarán el 100% de la variabilidad original.

Observa que en problemas de __alta dimensión__, en los que el número de variables es mayor que el número de observaciones, PCA permite conseguir una __gran reducción de la dimensión__ sin ofrecer casi pérdida de información. __El coste, claro, está en la interpretabilidad de las nuevas variables__.

Por otro lado, más que un número al azar de componentes, lo habitual es elegir el número de componentes tal que nos permita explicar hasta un cierto porcentaje de la variabilidad original (lo habitual suele ser en torno al 95% pero depende de los datos). Sklearn tiene esto en cuenta, así que el parámetro `n_components` puede fijarse como un número de componentes, o, si es un número entre 0 y 1, se interpreta como un porcentaje de variabilidad a explicar. Por ejemplo, construyamos suficientes componentes para explicar el 95% de la variabilidad del conjunto de datos de iris.

In [ ]:
pca_iris = PCA(n_components=0.95)

T_iris = pca_iris.fit_transform(X_iris)  # scores
P_iris = pca_iris.components_.T          # loadings

P_iris

### <font color='D12828'> Ejercicio: </font>

1. Obtén las componentes principales suficientes para explicar el 90% de la variabilidad del conjunto de datos de cancer estandarizado. ¿Cuantas componentes son suficientes?
2. Construye un clasificador KNN con `n_neighbors=10` que utilice estas componentes principales
3. Obtén el error de clasificación 

In [ ]:
# Ejercicio


### <font color='D12828'> Ejercicio: </font>
4. Repite el proceso con el conjunto de datos no estandarizado. ¿Cual da mejor resultado?

In [ ]:
# Ejercicio


Geométricamente, quedarse con tantas componentes principales como el número de variables iniciales implica que el número de dimensiones, y la distancia entre las observaciones es la misma, es decir, no traería ningún cambio en los resultados de un algoritmo predictivo con respecto a usar los datos originales.

### <font color='D12828'> Ejercicio: </font>
1. En el conjunto de datos de boston estandarizado `X_boston_train_s`, construye una SVM de regresión con todas las variables y obtén el error cuadrático medio
2. Obtén tantas componentes principales como variables haya en el conjunto de datos boston estandarizado, y construye otra SVM con las componentes. Obtén el error cuadrático medio

In [ ]:
# Ejercicio


# Linear discriminant analysis

Otro tipo de proyección que ya conocereis es el linear discriminant analysis, utilizado en problemas de clasificación. La idea detrás de esta técnica es sencilla: construir unas nuevas variables en base a las originales en las que clasificar los datos sea más sencillo que con las variables originales. El número de componentes que se pueden calcular aquí es min(n_classes - 1, n_var)). Usemos esta técnica con los datos de cancer. Dado que en estos datos tenemos 30 variables pero solo 2 categorías, proyectaremos los datos en 1 dimensión.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()

X_bc_train_lda = lda.fit_transform(X_bc_train_s, y_bc_train)

plt.scatter(X_bc_train_lda[:,0], [0]*455, c=y_bc_train)

In [ ]:
# Predecir en el conjunto de test
y_test_pred = lda.predict(X_bc_test_s)

# Error 
error = accuracy_score(y_bc_test, y_test_pred)
print(f'Accuracy: {error}')

# Final
___
Con esto termina nuestra introducción a las técnicas de reducción de la dimensión, ¡espero que hayais dsifrutado!

![image.png](plots/fin_svm_2.png)
